In [22]:
import warnings
warnings.filterwarnings("ignore")
from gensim.models import KeyedVectors, FastText
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pickle
import operator

# import natural language toolkit
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation #, strip_numbers
from nltk.corpus   import stopwords
from nltk.tokenize import word_tokenize

import string

ImportError: cannot import name 'strip_numbers' from 'gensim.parsing.preprocessing' (C:\Users\zivaurbancic\.conda\envs\envirolens\lib\site-packages\gensim\parsing\preprocessing.py)

In [2]:
from modules.library.postgresql import PostgresQL
from modules.library.word_models import WordModels
from modules.library.document_similarity import DocumentSimilarity
from modules.library.document_models import DocumentModels

# Accessing the database

Using the module PostgresQL we will load the documents from our database and store them in a list 'documents'.

In [3]:
# connect to the database
pg = PostgresQL() 
pg.connect(database="eurlex_environment_only", user="postgres", password="dbpass")

In [4]:
documents = pg.execute("""
    SELECT * FROM documents;
""")

# Word embeddnig model

In this section we will show how to use the module WordModels to load or train a word embedding model. In order to avoid runnig a time consuming commands, we will comment them out and use pickle instead to store and access already loaded or trained models. 

## Training a word embedding model

In case we want to use pre-trained models to train another model on our data, we can use a method in module WordModels called 'train' in the following way:

In [5]:
texts = ["dogs like chasing cats", "cats like chasing mice", "mice eat cheese", "cheese has holes", "earth flat", "nobody knows working"]

sample_model = WordModels()
sample_model.train(texts, size=300, window=1, min_count=1, epochs=10) # texts is a list of stripped strings

True

In [6]:
sample_model.get_embedding().vocab.keys()

dict_keys(['d', 'o', 'g', 's', ' ', 'l', 'i', 'k', 'e', 'c', 'h', 'a', 'n', 't', 'm', 'r', 'f', 'b', 'y', 'w'])

In [7]:
type(sample_model)

modules.library.word_models.WordModels

In [8]:
type(sample_model.get_embedding())

gensim.models.keyedvectors.FastTextKeyedVectors

## Loading a word embedding model

In case model has already been trained either by the user or has been provided from another source, module WordModels enables us to load it and use it.

In [9]:
wiki_en_path = '../data/fasttext/wiki.en.align.vec'
wiki_en_model = WordModels()
wiki_en_model.load(wiki_en_path)

In [10]:
# with open("wwe.pkl", "wb") as f:
#     pickle.dump(wiki_en_model, f, protocol=-1)

In [11]:
wiki_bin_path ='../data/fasttext/wiki.en.bin'
wiki_bin_model = WordModels()

In [12]:
# wiki_bin_model.load(wiki_bin_path)

In [13]:
wiki_bin_model.load(wiki_bin_path, model_type='fasttext')

In [14]:
# with open("wwe.pkl", "rb") as f:
#     wiki_en_model = pickle.load(f)

In [15]:
# path_law = '../data/Law2Vec/Law2Vec.200d.txt'
# law_model = WordModels()

In [16]:
# law_model.load(path_law)

In [17]:
# with open("law_we.pkl", "wb") as f:
#     pickle.dump(law_model, f, protocol = -1)

In [18]:
# with open("law_we.pkl", "rb") as f:
#     law_model = pickle.load(f)

## Embedding documents

We have some documents saved in a list 'documents'. Let's embed them using DocumentModels module.

In [19]:
document_texts = [doc['document_text'] for doc in documents if len(doc['document_text'])>0]

In [20]:
stop_words = stopwords.words('english') + list(string.punctuation)
document_model = DocumentModels(wiki_en_model.get_model().wv, document_texts, stop_words)

In [21]:
document_model.embed_documents()

NameError: name 'strip_numbers' is not defined

In [ ]:
embedding = document_model.get_embedding()

In [ ]:
u, s, vh = np.linalg.svd(embedding, full_matrices=True)

In [ ]:
first_singular_vector = vh[0,:]

In [ ]:
for i in range(len(document_texts)):
    embedding[i, :] = embedding[i, :] - first_singular_vector

## Document Similarity Analysis

In order to do analysis on a corpus of documents we will use module 'DocumentSimilarity'. Below are some examples of use:

In [ ]:
document_embedding = document_model.get_embedding()
ds = DocumentSimilarity(document_embedding)

In [ ]:
ds.euclid_similarity(document_embedding[0], document_embedding[1])

In [ ]:
list_of_neighbors = ds.k_nearest_neighbors(document_embedding[43877])

In [ ]:
list_of_neighbors

In [ ]:
for i in list_of_neighbors:
    print(documents[i]['document_title'])
    print(" ")